In [92]:
import pandas as pd
import gdown
import ast
import json
import numpy as np
import requests
from datetime import datetime

In [3]:
url = '1OxQupfqEt36O2GwnP7gL-2Kq9pdR5Eii'
output = "vacancies.xlsx"
gdown.download('https://drive.google.com/uc?export=download&id=' + url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1OxQupfqEt36O2GwnP7gL-2Kq9pdR5Eii
To: /content/vacancies.xlsx
173MB [00:01, 172MB/s]


'vacancies.xlsx'

In [4]:
df = pd.read_excel("vacancies.xlsx")

In [5]:
# Переводим текст в словари
df['area'] = df['area'].apply(lambda x: ast.literal_eval(x))
df['salary'] = df['salary'].apply(lambda x: ast.literal_eval(x) if not pd.isnull(x) else None)

In [141]:
# Разбиваем словари по столбцам
df['area_id'] = df['area'].apply(lambda x: x['id'])
df['area_name'] = df['area'].apply(lambda x: x['name'])
df['salary_from'] = df['salary'].apply(lambda x: x['from'] if not pd.isnull(x) else None)
df['salary_to'] = df['salary'].apply(lambda x: x['to'] if not pd.isnull(x) else None)
df['salary_currency'] = df['salary'].apply(lambda x: x['currency'] if not pd.isnull(x) else None)
df['salary_gross'] = df['salary'].apply(lambda x: x['gross'] if not pd.isnull(x) else None)

In [57]:
# Получаем все города и их parent_id
req = requests.get('https://api.hh.ru/areas')  # Посылаем запрос к API
data = req.content.decode()  # Декодируем его ответ, чтобы Кириллица отображалась корректно
req.close()
jsonObj = json.loads(data)
cities, countries_name, areas_name = [], [], []
for i in range(len(jsonObj)):
    for area in jsonObj[i]['areas']:
      if area['areas'] == []:
        cities.append(area)
        countries_name.append(jsonObj[i]['name'])
        areas_name.append(area['name'])
      else:
        for city in area['areas']:
          cities.append(city)
          countries_name.append(jsonObj[i]['name'])
          areas_name.append(area['name'])
df_cities = pd.DataFrame(cities)
df_cities['country_name'] = countries_name
df_cities['areas_name'] = areas_name

In [134]:
# Переводим текст в формат даты
df['published_at'] = df['published_at'].apply(lambda x: datetime.strptime(x,  "%Y-%m-%dT%H:%M:%S%z"))

In [42]:
# Переименовываем столбец
df_cities['area_id'] = df_cities['id']
df_cities = df_cities.drop('id', 1)

In [43]:
# Присваиваем каждой строке имя региона и страны
df['areas_name'] = df['area_id'].map(df_cities.set_index('area_id')['areas_name'])
df['country_name'] = df['area_id'].map(df_cities.set_index('area_id')['country_name'])

In [ ]:
# Начало описательной статистики
pd.set_option('float_format', '{:.0f}'.format)
df[(df['salary_currency'] == 'RUR') & (df['country_name'] == 'Россия')].describe()

In [ ]:
df.count()

In [ ]:
df['published_at'].min()

In [142]:
table = df.groupby(['country_name']).size().reset_index(name='count')
table
# table.to_excel('table.xlsx', index = False)

,country_name,count
0,Азербайджан,355
1,Беларусь,12560
2,Грузия,72
3,Другие регионы,976
4,Казахстан,13069
5,Кыргызстан,688
6,Россия,127710
7,Узбекистан,2589
8,Украина,6897
